In [1]:
import requests
import json
from itertools import product
import time

In [4]:
from datetime import timedelta
import requests
import json
from itertools import product

# Функция для форматирования времени из секунд в часы и минуты
def format_duration(seconds):
    delta = timedelta(seconds=seconds)
    hours, remainder = divmod(delta.seconds, 3600)
    minutes, _ = divmod(remainder, 60)
    return f"{hours} часов {minutes} минут"

# Функция для получения списка всех станций в указанных городах
def get_stations_in_cities(data, city_codes):
    stations = []
    for country in data.get("countries", []):
        for region in country.get("regions", []):
            for settlement in region.get("settlements", []):
                if settlement.get("codes", {}).get("yandex_code") in city_codes:
                    for station in settlement.get("stations", []):
                        station_type = station.get("station_type", "unknown")  # Используем station_type
                        # Оставляем только нужные типы станций
                        if station_type in ["train_station", "airport", "bus_station"]:
                            stations.append({
                                "code": station["codes"]["yandex_code"],
                                "title": station["title"],
                                "type": station_type  # Тип станции
                            })
    return stations

# Функция для поиска расписания между двумя станциями
def get_schedule_between_stations(from_station, to_station, date):
    url_search = "https://api.rasp.yandex.net/v3.0/search/"
    params = {
        "apikey": "95e48825-709d-4296-952b-eed993562fd5",
        "lang": "ru_RU",
        "format": "json",
        "from": from_station,
        "to": to_station,
        "date": date  # Дата в формате "YYYY-MM-DD"
    }

    response = requests.get(url_search, params=params)
    if response.status_code == 200:
        data = response.json()
        if data.get("segments"):  # Если есть расписание
            return data["segments"][0]["duration"]  # Возвращаем duration первого сегмента
    return None

# Функция для получения информации о нитке (thread) по её uid
def get_thread_info(uid):
    url_thread = "https://api.rasp.yandex.net/v3.0/thread/"
    params = {
        "apikey": "95e48825-709d-4296-952b-eed993562fd5",
        "lang": "ru_RU",
        "format": "json",
        "uid": uid
    }

    response = requests.get(url_thread, params=params)
    if response.status_code == 200:
        return response.json()
    return None

# Функция для поиска расписаний по станции
def get_schedule_for_station(station_code, date):
    url_schedule = "https://api.rasp.yandex.net/v3.0/schedule/"
    params = {
        "apikey": "95e48825-709d-4296-952b-eed993562fd5",
        "lang": "ru_RU",
        "format": "json",
        "station": station_code,
        "date": date
    }

    response = requests.get(url_schedule, params=params)
    if response.status_code == 200:
        data = response.json()
        return data.get("schedule", [])
    return []

# Основной код
if __name__ == "__main__":
    # Запрос списка всех станций
    url_stations_list = "https://api.rasp.yandex.net/v3.0/stations_list/"
    params_1 = {
        "apikey": "95e48825-709d-4296-952b-eed993562fd5",
        "lang": "ru_RU",
        "format": "json"
    }

    response_stations_list = requests.get(url_stations_list, params=params_1)
    if response_stations_list.status_code != 200:
        print("Ошибка при получении списка станций")
        exit()

    data_1 = response_stations_list.json()

    # Пользователь вводит коды городов
    city_code_A = input("Введите код города А (например, c195 - Ульяновск): ")
    city_code_B = input("Введите код города Б (например, c11155 - Димитровград): ")

    # Получаем списки станций в указанных городах
    print("Получаем списки станций...")
    stations_A = get_stations_in_cities(data_1, [city_code_A])
    stations_B = get_stations_in_cities(data_1, [city_code_B])

    if not stations_A or not stations_B:
        print("Станции в указанных городах не найдены.")
        exit()

    # Формируем словари для городов
    city_A = {}
    city_B = {}

    for station in stations_A:
        station_type = station["type"]
        if station_type not in city_A:
            city_A[station_type] = []
        city_A[station_type].append({
            "code": station["code"],
            "title": station["title"]  # Сохраняем название станции
        })

    for station in stations_B:
        station_type = station["type"]
        if station_type not in city_B:
            city_B[station_type] = []
        city_B[station_type].append({
            "code": station["code"],
            "title": station["title"]  # Сохраняем название станции
        })

    # Пользователь вводит дату
    date = input("Введите дату в формате YYYY-MM-DD (например, 2025-03-28): ")

    # Словарь для сохранения результатов
    routes = {}

    # 1. Поиск прямых рейсов между основными станциями городов А и Б
    print("Поиск рейсов между станциями городов А и Б...")
    for station_type in city_A:
        if station_type in city_B:
            pairs = list(product(city_A[station_type], city_B[station_type]))
            for pair in pairs:
                from_station = pair[0]
                to_station = pair[1]

                print(f"Поиск рейсов от {from_station['title']} ({from_station['code']}) до {to_station['title']} ({to_station['code']})...")

                duration = get_schedule_between_stations(from_station["code"], to_station["code"], date)
                if duration:
                    route_name = f"{from_station['title']} ({from_station['code']}) - {to_station['title']} ({to_station['code']})"
                    routes[route_name] = duration
                    print(f"Найден рейс! Продолжительность: {format_duration(duration)}.")
                else:
                    print("Рейсов нет.")

Введите код города А (например, c195 - Ульяновск): c195
Введите код города Б (например, c11155 - Димитровград): c11155
Получаем списки станций...
Введите дату в формате YYYY-MM-DD (например, 2025-03-28): 2025-03-28
Поиск рейсов между станциями городов А и Б...
Поиск рейсов от Верхняя Терраса (s9606439) до Димитровград (s9606447)...
Найден рейс! Продолжительность: 1 часов 21 минут.
Поиск рейсов от Ульяновск-Центр. (s9606620) до Димитровград (s9606447)...
Найден рейс! Продолжительность: 1 часов 48 минут.
